# Data Structures for Data Science (Python) — Jupyter Notebook

**Last updated:** 2026-02-07

This notebook is a hands-on lab that teaches the most useful data structures for data science work in Python.

**How to use this notebook**
- Read the concept cells, then run the code cells.
- After each section, try the short exercises.
- This notebook assumes basic Python (variables, functions, loops).


## Learning goals
By the end, you will be able to:
- Choose the right structure for a task (list vs dict vs set vs array)
- Explain time/space tradeoffs using Big-O intuition
- Use stacks, queues, heaps, and deques for common DS workflows
- Work confidently with NumPy arrays and pandas Series/DataFrames
- Recognize when graphs/trees are the right mental model


## 0) Setup
We’ll import a few standard libraries used throughout.

In [1]:
import math
import random
import time
from collections import Counter, defaultdict, deque
import heapq

import numpy as np
import pandas as pd


## 1) Big-O intuition (for data science)
In DS work, performance issues often show up when:
- A loop grows with data size (N)
- Nested loops (N²) appear unintentionally
- We repeatedly search a list instead of using a hash-based structure (dict/set)

### Common operations (rule-of-thumb)
- **List**: append O(1) amortized, search (`x in list`) O(N)
- **Dict/Set**: insert/lookup O(1) average
- **Sorting**: O(N log N)

We’ll do a quick timing comparison: list membership vs set membership.


In [2]:
# Timing: list membership vs set membership
N = 200_000
items = list(range(N))
items_set = set(items)

targets = [random.randrange(N) for _ in range(50_000)]

t0 = time.time()
hits_list = sum(1 for x in targets if x in items)   # O(N) membership each time
t1 = time.time()

t2 = time.time()
hits_set = sum(1 for x in targets if x in items_set)  # O(1) average membership
t3 = time.time()

print("hits_list:", hits_list, "time:", round(t1-t0, 4), "sec")
print("hits_set :", hits_set,  "time:", round(t3-t2, 4), "sec")


hits_list: 50000 time: 48.7773 sec
hits_set : 50000 time: 0.0075 sec


### Exercise 1
1. Increase `N` to 1,000,000 and rerun.
2. What happens to the timing gap? Why?


## 2) Python lists: flexible sequences
Lists are great for:
- Collecting records
- Maintaining order
- Iterating

Less great for:
- Frequent membership checks
- Deleting from the middle repeatedly


In [3]:
# Basic list patterns
values = [3, 1, 4, 1, 5]
values.append(9)
values.extend([2, 6])
print(values)

# Common DS pattern: build and transform
squared = [x*x for x in values if x % 2 == 1]
squared


[3, 1, 4, 1, 5, 9, 2, 6]


[9, 1, 1, 25, 81]

### Slicing and copying
Be careful: `a = b` does not copy a list; it references the same object.

In [4]:
a = [1, 2, 3]
b = a          # same object
c = a[:]       # shallow copy
a.append(4)

print("a:", a)
print("b:", b, "(same as a)")
print("c:", c, "(copy)")


a: [1, 2, 3, 4]
b: [1, 2, 3, 4] (same as a)
c: [1, 2, 3] (copy)


### Exercise 2
Given a list of numbers, create a new list that contains only values above the mean.


In [5]:
nums = [random.randint(0, 100) for _ in range(30)]
mean_val = sum(nums) / len(nums)
above_mean = [x for x in nums if x > mean_val]
mean_val, nums, above_mean


(57.36666666666667,
 [63,
  65,
  36,
  61,
  66,
  13,
  2,
  80,
  78,
  98,
  32,
  22,
  100,
  98,
  90,
  14,
  4,
  26,
  48,
  79,
  51,
  31,
  69,
  22,
  89,
  42,
  80,
  73,
  94,
  95],
 [63, 65, 61, 66, 80, 78, 98, 100, 98, 90, 79, 69, 89, 80, 73, 94, 95])

## 3) Tuples: lightweight, immutable records
Tuples are good for:
- Fixed-size records (like a row)
- Dictionary keys (when you need a composite key)
- Safer data that shouldn’t change


In [ ]:
row = ("TX", "Dallas", 1300000)
state, city, population = row
state, city, population


## 4) Dictionaries: fast lookup (hash tables)
Dictionaries are the workhorse for:
- Mapping IDs → records
- Counting categories
- Caching intermediate results

### Pattern: counting with dict / Counter


In [ ]:
text = "data science is science about data"
counts = Counter(text.split())
counts


In [ ]:
# Defaultdict for grouping
records = [
    {"dept": "Cardiology", "cost": 1200},
    {"dept": "Cardiology", "cost": 900},
    {"dept": "Oncology", "cost": 3000},
    {"dept": "Oncology", "cost": 2800},
    {"dept": "ER", "cost": 600},
]

by_dept = defaultdict(list)
for r in records:
    by_dept[r["dept"]].append(r["cost"])

by_dept, {k: sum(v)/len(v) for k,v in by_dept.items()}


### Exercise 3
Group patients by risk quadrant.
- Input: list of dicts with keys `patient_id` and `risk_quadrant`
- Output: dict mapping quadrant → list of patient_ids


In [ ]:
patients = [
    {"patient_id": "P001", "risk_quadrant": "High"},
    {"patient_id": "P002", "risk_quadrant": "Rising"},
    {"patient_id": "P003", "risk_quadrant": "High"},
    {"patient_id": "P004", "risk_quadrant": "Low"},
    {"patient_id": "P005", "risk_quadrant": "Rising"},
]

grouped = defaultdict(list)
for p in patients:
    grouped[p["risk_quadrant"]].append(p["patient_id"])

dict(grouped)


## 5) Sets: uniqueness + fast membership
Sets are ideal for:
- Deduplicating
- Fast membership checks
- Set operations: union, intersection, difference


In [ ]:
a = {"hypertension", "diabetes", "asthma"}
b = {"diabetes", "cad"}

print("union:", a | b)
print("intersection:", a & b)
print("difference:", a - b)


### Exercise 4
Given two lists of patient IDs (screened vs enrolled), compute:
- IDs screened but not enrolled
- IDs enrolled but not screened
- IDs in both


In [ ]:
screened = ["P001", "P002", "P003", "P010"]
enrolled  = ["P002", "P003", "P004"]

S = set(screened)
E = set(enrolled)

screened_not_enrolled = S - E
enrolled_not_screened = E - S
both = S & E

screened_not_enrolled, enrolled_not_screened, both


## 6) Deques: queues and sliding windows
`collections.deque` supports O(1) append/pop from both ends.
Common DS use cases:
- FIFO queue
- Moving/sliding windows
- Breadth-first search (BFS)


In [ ]:
q = deque()
q.append("A")
q.append("B")
q.append("C")

first = q.popleft()  # FIFO
print("popped:", first)
print("queue:", list(q))


In [ ]:
# Sliding window example: moving average
x = [random.randint(0, 10) for _ in range(20)]
k = 5

window = deque(maxlen=k)
avgs = []
for val in x:
    window.append(val)
    if len(window) == k:
        avgs.append(sum(window)/k)

x, avgs[:5], len(avgs)


## 7) Stacks: LIFO workflows
Python lists can act as stacks using `.append()` and `.pop()`.
Stacks appear in:
- Undo features
- Depth-first search (DFS)
- Parsing / parentheses matching


In [ ]:
stack = []
for ch in "ABCDE":
    stack.append(ch)

while stack:
    print(stack.pop(), end=" ")
print()


### Exercise 5: Parentheses matching
Write a function `is_balanced(s)` that returns True if parentheses are balanced.
Test on: `"(a+b) * (c+d)"`, `"(a+b))"`, `"((a+b)"`.


In [ ]:
def is_balanced(s: str) -> bool:
    stack = []
    for ch in s:
        if ch == "(":
            stack.append(ch)
        elif ch == ")":
            if not stack:
                return False
            stack.pop()
    return len(stack) == 0

tests = ["(a+b) * (c+d)", "(a+b))", "((a+b)"]
[(t, is_balanced(t)) for t in tests]


## 8) Heaps (priority queues)
Heaps support fast access to the smallest element.
Use cases:
- Top-K problems (most common in DS)
- Scheduling
- Streaming analytics

Python provides `heapq` (a min-heap).


In [ ]:
data = [random.randint(0, 1000) for _ in range(20)]
k = 5

top_k = heapq.nlargest(k, data)
bottom_k = heapq.nsmallest(k, data)

data, top_k, bottom_k


### Exercise 6: Top-K frequent items
Given a list of diagnosis codes, find the 3 most common.


In [ ]:
dx = ["I10", "E11.9", "I10", "I25.10", "E11.9", "I10", "E78.5", "E78.5", "E78.5", "I25.10"]
freq = Counter(dx)
top3 = freq.most_common(3)
freq, top3


## 9) NumPy arrays: fast numeric structure
NumPy arrays are:
- Homogeneous (single dtype)
- Vectorized (fast operations)
- Memory-efficient

Use arrays for numeric features, matrices, and heavy computation.


In [ ]:
arr = np.array([1, 2, 3, 4, 5], dtype=float)
arr * 2, arr.mean(), arr.std()


In [ ]:
# Broadcasting example: standardize features
X = np.random.randn(1000, 3) * np.array([10, 2, 1]) + np.array([50, 10, 0])
mu = X.mean(axis=0)
sigma = X.std(axis=0)

Z = (X - mu) / sigma
Z.mean(axis=0), Z.std(axis=0)


### Exercise 7
Create a 2D NumPy array with shape (100, 4) representing 100 patients and 4 features.
Compute column means and identify which feature has the largest variance.


In [ ]:
X = np.random.randn(100, 4)
col_means = X.mean(axis=0)
col_vars = X.var(axis=0)
largest_var_feature = int(np.argmax(col_vars))
col_means, col_vars, largest_var_feature


## 10) pandas Series and DataFrames: labeled data structures
pandas is optimized for:
- Tabular data
- Missing values
- Groupby/aggregate
- Joining/merging

Series = 1D with an index
DataFrame = 2D with row index + columns


In [ ]:
df = pd.DataFrame({
    "patient_id": ["P001","P002","P003","P004","P005"],
    "bp_sys": [130, 142, 118, 155, 140],
    "bp_dia": [82,  90,  76,  95,  88],
    "smoker": [0, 1, 0, 1, 0],
    "dept": ["Cardiology", "Cardiology", "ER", "ER", "Oncology"]
})

df


In [ ]:
# Common DS operations
df["pulse_pressure"] = df["bp_sys"] - df["bp_dia"]
df.sort_values("bp_sys", ascending=False)


In [ ]:
# Groupby aggregation
df.groupby("dept").agg(
    n=("patient_id", "count"),
    mean_sys=("bp_sys", "mean"),
    smoker_rate=("smoker", "mean"),
)


### Exercise 8
Filter to systolic BP ≥ 140. Then compute mean systolic BP by department for that subset.


In [ ]:
high = df[df["bp_sys"] >= 140]
high.groupby("dept")["bp_sys"].mean()


## 11) Sparse data (optional but useful)
Many DS problems have high-dimensional sparse features (e.g., bag-of-words).
In those cases, you often store data in a **sparse matrix** (SciPy) instead of a dense NumPy array.

We won’t import SciPy here, but the key concept:
- Dense: stores all zeros → wastes memory
- Sparse: stores only non-zeros → efficient


## 12) Graphs: relationships, networks, pathways
Graphs model:
- Social networks
- Knowledge graphs
- Patient referral networks
- Disease comorbidity networks

A simple representation is an **adjacency list** using a dict of lists.


In [ ]:
# Build an undirected graph with adjacency lists
graph = {
    "A": ["B", "C"],
    "B": ["A", "D"],
    "C": ["A", "D"],
    "D": ["B", "C", "E"],
    "E": ["D"]
}

# BFS: shortest path in an unweighted graph
def bfs_shortest_path(graph, start, goal):
    q = deque([(start, [start])])
    seen = {start}
    while q:
        node, path = q.popleft()
        if node == goal:
            return path
        for nbr in graph.get(node, []):
            if nbr not in seen:
                seen.add(nbr)
                q.append((nbr, path + [nbr]))
    return None

bfs_shortest_path(graph, "A", "E")


### Exercise 9
Modify the graph to add a new node `F` connected to `C`.
Then find a shortest path from `F` to `E`.


In [ ]:
graph2 = {k: v.copy() for k,v in graph.items()}
graph2["F"] = ["C"]
graph2["C"] = graph2["C"] + ["F"]

bfs_shortest_path(graph2, "F", "E")


## 13) Trees (brief): hierarchical data
Trees appear in:
- File systems
- Taxonomies (ICD hierarchy)
- Decision trees / gradient boosting (model structure)

We’ll implement a tiny tree node structure and a depth-first traversal.


In [ ]:
class Node:
    def __init__(self, value, children=None):
        self.value = value
        self.children = children or []

# Simple tree
root = Node("root", [
    Node("A", [Node("A1"), Node("A2")]),
    Node("B", [Node("B1")]),
    Node("C")
])

def dfs_values(node):
    out = [node.value]
    for child in node.children:
        out.extend(dfs_values(child))
    return out

dfs_values(root)


## 14) Choosing the right structure: mini decision guide
- Need **order** and easy appends? → list
- Need **unique items** or fast membership? → set
- Need **key → value lookup**? → dict
- Need **FIFO queue / sliding window**? → deque
- Need **top-k** or priority? → heapq
- Need **fast numeric computation**? → NumPy array
- Need **labeled tabular data**? → pandas DataFrame
- Need **relationships/networks**? → graph (adjacency list)


## 15) Capstone mini-lab: frequent items + join + top-k
Scenario: You have encounters with diagnosis codes and want:
1) Counts per diagnosis
2) Join with a reference table (dx → category)
3) Top 5 diagnoses overall and by category


In [ ]:
encounters = pd.DataFrame({
    "encounter_id": range(1, 21),
    "patient_id": [f"P{random.randint(1,5):03d}" for _ in range(20)],
    "dx": random.choices(["I10","E11.9","I25.10","E78.5","J45.909","F32.9"], k=20)
})

dx_ref = pd.DataFrame({
    "dx": ["I10","E11.9","I25.10","E78.5","J45.909","F32.9"],
    "category": ["HTN", "Diabetes", "CAD", "Lipids", "Asthma", "Depression"]
})

encounters.head(), dx_ref


In [ ]:
# 1) counts per diagnosis
dx_counts = encounters["dx"].value_counts().rename_axis("dx").reset_index(name="n")
dx_counts


In [ ]:
# 2) join with reference
dx_counts2 = dx_counts.merge(dx_ref, on="dx", how="left")
dx_counts2


In [ ]:
# 3) Top 5 overall and by category
top5_overall = dx_counts2.sort_values("n", ascending=False).head(5)

by_cat = (encounters.merge(dx_ref, on="dx", how="left")
          .groupby("category")["dx"]
          .count()
          .sort_values(ascending=False)
         )

top5_overall, by_cat


### Capstone exercise
1) Compute the **top 2 diagnoses per category** (hint: groupby + value_counts, or groupby + apply).
2) For each patient, compute their **most common category**.

If you get stuck, tell me what you tried and I’ll help you debug it.


In [ ]:
# Starter cell for Capstone exercise (fill in)
# 1) Top 2 diagnoses per category

tmp = encounters.merge(dx_ref, on="dx", how="left")
tmp.head()

# Your solution here:
